In [7]:
import sys
import os as os
import numpy as np
import polars as pl
import pandas as pd
import geopandas as gpd
import json
import math
import joblib
import copy
import sqlite3
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error 
from pygam import LinearGAM, s

# from shapely.geometry import Point
# from shapely.ops import unary_union
# from dataclasses import dataclass, field
# from typing_extensions import List, Dict, Tuple

if 'scicore' in os.getcwd():
    path = '/scicore/home/krysiak/hocrau00/ondemand/OptimalPV_RH'
else:
    path = os.getcwd().split('\\src')[0]
    %load_ext rpy2.ipython

os.chdir(path)
from src.calibration_class import Calibration_Settings, Calibration



The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [8]:
name_dir_export_path_PY = 'c:/Models/OptimalPV_RH/data/calibration/calib_all_CH_bfs3'

In [9]:
%%R
name_dir_export_path_R <- "c:/Models/OptimalPV_RH/data/calibration/calib_all_CH_bfs3"

## approach 2

### data import

In [10]:
# weed out all inf or NaN values 
df = pd.read_csv(f'{name_dir_export_path_PY}/reg2_all_CH_bfs_df_approach2.csv')
df = df.dropna(subset=['BFS_NUMMER', 'TotalPower', 'elecpri_Rp_kWh', 'pvtarif_Rp_kWh', 'north_max_flaeche', 'south_max_flaeche', 'east_max_flaeche', 'west_max_flaeche'])
df.to_csv(f'{name_dir_export_path_PY}/reg2_all_CH_bfs_df_cleaned.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'c:/Models/OptimalPV_RH/data/calibration/calib_all_CH_bfs3/reg2_all_CH_bfs_df_approach2.csv'

In [ ]:
%%R
if (!requireNamespace("readr", quietly = TRUE)) install.packages("readr")
if (!requireNamespace("dplyr", quietly = TRUE)) install.packages("dplyr")
if (!requireNamespace("fixest", quietly = TRUE)) install.packages("fixest")
if (!requireNamespace("tibble", quietly = TRUE)) install.packages("tibble")
if (!requireNamespace("jsonlite", quietly = TRUE)) install.packages("jsonlite")
if (!requireNamespace("tidyr", quietly = TRUE)) install.packages("tidyr")
if (!requireNamespace("ggplot2", quietly = TRUE)) install.packages("ggplot2")
# if (!requireNamespace("purr", quietly = TRUE)) install.packages("purr")
if (!requireNamespace("tidyverse", quietly = TRUE)) install.packages("tidyverse")
if (!requireNamespace("randomForest", quietly = TRUE)) install.packages("randomForest")

library(readr)
library(dplyr)
library(fixest)
library(tibble)
library(jsonlite)
library(tidyr)
library(ggplot2)
# library(purrr)
library(tidyverse)
library(randomForest)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v forcats   1.0.1     v purrr     1.1.0
v lubridate 1.9.4     v stringr   1.5.2
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter()  masks stats::filter()
x purrr::flatten() masks jsonlite::flatten()
x dplyr::lag()     masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors



Attache Paket: 'dplyr'

Die folgenden Objekte sind maskiert von 'package:stats':

    filter, lag

Die folgenden Objekte sind maskiert von 'package:base':

    intersect, setdiff, setequal, union

randomForest 4.7-1.2
Type rfNews() to see new features/changes/bug fixes.

Attache Paket: 'randomForest'

Das folgende Objekt ist maskiert 'package:ggplot2':

    margin

Das folgende Objekt ist maskiert 'package:dplyr':

    combine

In addition: There were 13 warnings (use warnings() to see them)


In [ ]:
%%R
# train / test split-------------
# train_test_split = 0.7

# import -------------
pq_files = list.files(name_dir_export_path_R, pattern = ".parquet", full.names = TRUE)

# df <- tibble(read_csv(paste0(name_dir_export_path_R, "/reg2_all_CH_bfs_df_cleaned.csv"), locale = locale(encoding = "UTF-8")))
df <- tibble(read_csv(paste0(name_dir_export_path_R, "/reg2_all_CH_bfs_df_cleaned.csv")))


df <- df %>% 
  mutate(
    GBAUJ    = as_factor(GBAUJ),
    GKLAS    = as_factor(GKLAS),
    GSTAT    = as_factor(GSTAT),
    GWAERZH1 = as_factor(GWAERZH1),
    GENH1    = as_factor(GENH1))

idx_heatpump <- df$GWAERZH1 %in% c('7410', '7411')
df$GWAERZH1_str <- 'no_heatpump'
df$GWAERZH1_str[idx_heatpump] <- 'heatpump'
df$GWAERZH1_str <- as_factor(df$GWAERZH1_str)


# split training & test data -------------
split_train_test <- function(df_func, df_sub_type, split_ratio = 0.7, seed = 42) {
  set.seed(seed)
  train_indices <- sample(1:nrow(df_func), size = split_ratio * nrow(df_func))
  df_train_func <- df_func[train_indices, ]
  df_test_func <- df_func[-train_indices, ]
  
  if (df_sub_type == "train"){
    df_return = df_train_func
  }
  else if (df_sub_type == "test"){
    df_return = df_test_func
  }
  else {
    stop("df_sub_type must be either 'train' or 'test'")
  } 
  return(df_return)
}

df_train <- split_train_test(df, "train" )
df_test <-  split_train_test(df, "test"  )


# filter: only residential buildings -------------
idx_GKLAS_res3plus <- df$GKLAS %in% c("1110", "1121", "1122")
df__res3plus <- df %>% filter(idx_GKLAS_res3plus)
df_train_res3plus <- split_train_test(df__res3plus, "train" )
df_test_res3plus <-  split_train_test(df__res3plus, "test"  )


idx_GKLAS_res1to2 <- df$GKLAS %in% c("1110", "1121")
df__res1to2 <- df %>% filter(idx_GKLAS_res1to2)
df_train_res1to2 <- split_train_test(df__res1to2, "train" )
df_test_res1to2 <-  split_train_test(df__res1to2, "test"  )

idx_kwpmax20 <- df$TotalPower < 20
df__kwpmax20 <- df %>% filter(idx_kwpmax20 & idx_GKLAS_res1to2)
df_train_kwpmax20 <- split_train_test(df__kwpmax20, "train" )
df_test_kwpmax20 <-  split_train_test(df__kwpmax20, "test"  )


calc_inst_flaeche <- function(row, sett) {
  # Convert to named list for convenience
  r <- as.list(row)
  
  directions <- c("south", "west", "east", "north")
  
  if (r$TotalPower > 0 && r$FLAECHE_total > 0) {
    
    inst_total_flaeche <- r$TotalPower * sett$TECspec_kWpeak_per_m2 * sett$TECspec_share_roof_area_available
    
    # Initialize ratios
    for (direction in directions) {
      r[[paste0(direction, "_max_ratio")]] <- 0
    }
    
    # Compute ratios by direction
    for (direction in directions) {
      fla_col  <- paste0(direction, "_max_flaeche")
      ratio_col <- paste0(direction, "_max_ratio")
      
      if (r[[fla_col]] <= 0) next
      
      max_inst_flaeche <- r[[fla_col]] * sett$TECspec_share_roof_area_available
      
      if (inst_total_flaeche <= 0) break
      
      if (inst_total_flaeche <= max_inst_flaeche) {
        r[[ratio_col]] <- inst_total_flaeche / max_inst_flaeche
        inst_total_flaeche <- 0
      } else {
        r[[ratio_col]] <- 1
        inst_total_flaeche <- inst_total_flaeche - max_inst_flaeche
      }
    }
    
  } else {
    # If no PV installation
    for (direction in directions) {
      r[[paste0(direction, "_max_ratio")]] <- 0
    }
  }
  
  return(unlist(r))
}

df_pvroof_ratio <- as.data.frame(t(apply(df, 1, calc_inst_flaeche, sett = sett)))
df_pvroof_ratio <- type.convert(df_pvroof_ratio, as.is = TRUE)

ratio_cols <- c("south_max_ratio", "west_max_ratio", "east_max_ratio", "north_max_ratio")
pvroof_ratio20to70_idx <- apply(df_pvroof_ratio[ratio_cols], 1, function(x) {
  any(x > 0.2 & x < 0.7)
})
df_pvroof_ratio20to70 <- df_pvroof_ratio[pvroof_ratio20to70_idx, ]


# export training & test data ------------
write_csv(df_train, paste0(getwd(), "/data/calibration/calib_all_CH_bfs/reg2_df_train.csv"))
write_csv(df_test, paste0(getwd(), "/data/calibration/calib_all_CH_bfs/reg2_df_test.csv"))

# write_csv(df_train_res3plus,  paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_train_res3plus.csv"))
# write_csv(df_test_res3plus,   paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_test_res3plus.csv"))
# write_csv(df_train_res1to2,   paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_train_res1to2.csv"))
# write_csv(df_test_res1to2,    paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_test_res1to2.csv" ))
# write_csv(df_train_kwpmax20,  paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_train_kwpmax20.csv"))
# write_csv(df_test_kwpmax20,   paste0(getwd(),  "/data/calibration/calib_all_CH_bfs/reg2_df_test_kwpmax20.csv" ))


Rows: 73549 Columns: 20
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (20): EGID, year, BFS_NUMMER, xtf_id, n_DF_UID, GAREA, GBAUJ, GKLAS, GST...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Error in (function (expr, envir = parent.frame(), enclos = if (is.list(envir) ||  : 
  object 'sett' not found


RInterpreterError: Failed to parse and evaluate line '# train / test split-------------\n# train_test_split = 0.7\n\n# import -------------\npq_files = list.files(name_dir_export_path_R, pattern = ".parquet", full.names = TRUE)\n\n# df <- tibble(read_csv(paste0(name_dir_export_path_R, "/reg2_all_CH_bfs_df_cleaned.csv"), locale = locale(encoding = "UTF-8")))\ndf <- tibble(read_csv(paste0(name_dir_export_path_R, "/reg2_all_CH_bfs_df_cleaned.csv")))\n\n\ndf <- df %>% \n  mutate(\n    GBAUJ    = as_factor(GBAUJ),\n    GKLAS    = as_factor(GKLAS),\n    GSTAT    = as_factor(GSTAT),\n    GWAERZH1 = as_factor(GWAERZH1),\n    GENH1    = as_factor(GENH1))\n\nidx_heatpump <- df$GWAERZH1 %in% c(\'7410\', \'7411\')\ndf$GWAERZH1_str <- \'no_heatpump\'\ndf$GWAERZH1_str[idx_heatpump] <- \'heatpump\'\ndf$GWAERZH1_str <- as_factor(df$GWAERZH1_str)\n\n\n# split training & test data -------------\nsplit_train_test <- function(df_func, df_sub_type, split_ratio = 0.7, seed = 42) {\n  set.seed(seed)\n  train

## OLS regression

In [13]:
%%R
df_test <-          tibble(read_csv(paste0(name_dir_export_path_R, "/df_test.csv")))
df_test_res3plus <- tibble(read_csv(paste0(name_dir_export_path_R, "/df_test_res3plus.csv")))
df_test_res1to2 <-  tibble(read_csv(paste0(name_dir_export_path_R, "/df_test_res1to2.csv")))

fe21 <- feols(
  TotalPower ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) +
    west_max_flaeche + I(west_max_flaeche^2) +
    south_max_flaeche + I(south_max_flaeche^2) +
    east_max_flaeche + I(east_max_flaeche^2) |
    BFS_NUMMER + year,
  data = df_train
)
df_test$pred_fe21 <- predict(fe21, newdata = df_test)

# only residential buildings of small size
fe30 <-  feols(
  TotalPower ~ 
    elecpri_Rp_kWh +
    pvtarif_Rp_kWh +
    GBAUJ + 
    GKLAS +  
    # GSTAT + # <- problematic row
    GWAERZH1_str + 
    west_max_flaeche + I(west_max_flaeche^2) +
    south_max_flaeche + I(south_max_flaeche^2) +
    east_max_flaeche + I(east_max_flaeche^2) |
    BFS_NUMMER + year,
  data = df_train
)
df_test$pred_fe30 <- predict(fe30, newdata = df_test)

fe31 <- feols(
  TotalPower ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + 
    GBAUJ + 
    GKLAS +  
    # GSTAT + # <- problematic row
    GWAERZH1_str + 
    west_max_flaeche + I(west_max_flaeche^2) +
    south_max_flaeche + I(south_max_flaeche^2) +
    east_max_flaeche + I(east_max_flaeche^2) |
    BFS_NUMMER + year,
  data = df_train_res3plus
)
df_test_res3plus$pred_fe31 <- predict(fe31, newdata = df_test_res3plus) 

fe32 <- feols(
  TotalPower ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + 
    GBAUJ + 
    GKLAS +  
    # GSTAT + # <- problematic row
    GWAERZH1_str + 
    west_max_flaeche + I(west_max_flaeche^2) +
    south_max_flaeche + I(south_max_flaeche^2) +
    east_max_flaeche + I(east_max_flaeche^2) |
    BFS_NUMMER + year,
  data = df_train_res1to2
)
df_test_res1to2$pred_fe32 <- predict(fe32, newdata = df_test_res1to2)

fe40 <- feols(
  TotalPower ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + 
    GBAUJ + 
    GKLAS +  
    # GSTAT + # <- problematic row
    GWAERZH1_str + 
    south_max_flaeche + I(south_max_flaeche^2) +
    FLAECHE_total + I(FLAECHE_total^2) |
    BFS_NUMMER + year,
  data = df_train_res3plus
)
df_test_res3plus$pred_fe40 <- predict(fe40, newdata = df_test_res3plus)

fe41 <- feols(
  TotalPower ~ 
    elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
    pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + 
    GBAUJ + 
    GKLAS +  
    # GSTAT + # <- problematic row
    GWAERZH1_str + 
    south_max_flaeche + I(south_max_flaeche^2) +
    FLAECHE_total + I(FLAECHE_total^2) |  
    BFS_NUMMER + year,
  data = df_train_res1to2
)
df_test_res1to2$pred_fe41 <- predict(fe41, newdata = df_test_res1to2)


write_csv(df_test,          paste0(name_dir_export_path_R, "/reg2_df_test.csv"))
write_csv(df_test_res3plus, paste0(name_dir_export_path_R, "/reg2_df_test_res3plus.csv"))
write_csv(df_test_res1to2,  paste0(name_dir_export_path_R, "/reg2_df_test_res1to2.csv"))


Rows: 22513 Columns: 25
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (1): GWAERZH1_str
dbl (24): EGID, year, BFS_NUMMER, xtf_id, n_DF_UID, GAREA, GBAUJ, GKLAS, GST...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 22513 Columns: 25
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (1): GWAERZH1_str
dbl (24): EGID, year, BFS_NUMMER, xtf_id, n_DF_UID, GAREA, GBAUJ, GKLAS, GST...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 20238 Columns: 25
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (1): GWAERZH1_str
dbl (24): EGID, year, BFS_NUMMER, xtf_id, n_DF_UID, GAREA, GBAUJ, GKLAS, GST...

i Use `spec()` to re

NOTE: 62/0 fixed-effect singletons were removed (62 observations).
The variables 'elecpri_Rp_kWh', 'I(I(elecpri_Rp_kWh^2))', 'pvtarif_Rp_kWh' and
'I(I(pvtarif_Rp_kWh^2))' have been removed because of collinearity (see
$collin.var).
NOTE: 62/0 fixed-effect singletons were removed (62 observations).
The variables 'elecpri_Rp_kWh' and 'pvtarif_Rp_kWh' have been removed because of
collinearity (see $collin.var).
NOTE: 62/0 fixed-effect singletons were removed (62 observations).
The variables 'elecpri_Rp_kWh', 'I(I(elecpri_Rp_kWh^2))', 'pvtarif_Rp_kWh' and
'I(I(pvtarif_Rp_kWh^2))' have been removed because of collinearity (see
$collin.var).
NOTE: 54/0 fixed-effect singletons were removed (54 observations).
The variables 'elecpri_Rp_kWh', 'I(I(elecpri_Rp_kWh^2))', 'pvtarif_Rp_kWh' and
'I(I(pvtarif_Rp_kWh^2))' have been removed because of collinearity (see
$collin.var).
NOTE: 62/0 fixed-effect singletons were removed (62 observations).
The variables 'elecpri_Rp_kWh', 'I(I(elecpri_Rp_kWh^2))',

### archived
archived models - no longer used (outperformed by other regressions)

In [ ]:
# %%R
# # fit regression model
# fe11 <- feols( TotalPower ~ elecpri_Rp_kWh + pvtarif_Rp_kWh + west_max_flaeche + south_max_flaeche + east_max_flaeche, data = df_train )
# fe12 <- feols( TotalPower ~ elecpri_Rp_kWh + pvtarif_Rp_kWh + west_max_flaeche + south_max_flaeche + east_max_flaeche | year, data = df_train    )

# fe13 <- feols( TotalPower ~
#     elecpri_Rp_kWh + pvtarif_Rp_kWh +
#     west_max_flaeche + south_max_flaeche + east_max_flaeche |
#     BFS_NUMMER,
#     data = df_train
#     )
# df_test$pred_fe13 <- predict(fe13, newdata = df_test)

# fe14 <- feols( TotalPower ~
#     elecpri_Rp_kWh + pvtarif_Rp_kWh +
#     west_max_flaeche + south_max_flaeche + east_max_flaeche | 
#     BFS_NUMMER + year,
#     data = df_train
#     )
# df_test$pred_fe14 <- predict(fe14, newdata = df_test)



# fe24a <- feols(log(TotalPower) ~ elecpri_Rp_kWh + pvtarif_Rp_kWh +west_max_flaeche + south_max_flaeche + east_max_flaeche, data = df_train)
# fe24b <- feols( log(TotalPower) ~  elecpri_Rp_kWh + pvtarif_Rp_kWh + west_max_flaeche + south_max_flaeche + east_max_flaeche |  year,  data = df_train )

# fe24c<- feols(
#     log(TotalPower) ~ 
#       elecpri_Rp_kWh + pvtarif_Rp_kWh +
#       west_max_flaeche + south_max_flaeche + east_max_flaeche | 
#       BFS_NUMMER, 
#     data = df_train
# )
# df_test$pred_fe24c <- predict(fe24c, newdata = df_test)

# fe24d<- feols(
#     log(TotalPower) ~ 
#       elecpri_Rp_kWh + pvtarif_Rp_kWh +
#       west_max_flaeche + south_max_flaeche + east_max_flaeche | 
#       BFS_NUMMER + year,
#     data = df_train
# )
# df_test$pred_fe24d <- predict(fe24d, newdata = df_test)

# fe25a<- feols(log(TotalPower) ~ elecpri_Rp_kWh + pvtarif_Rp_kWh +west_max_flaeche + south_max_flaeche + east_max_flaeche,data = df_train)
# fe25b<- feols( log(TotalPower) ~ elecpri_Rp_kWh + pvtarif_Rp_kWh + west_max_flaeche + south_max_flaeche + east_max_flaeche |  year, data = df_train)
# fe25c<- feols(
#     log(TotalPower) ~
#       elecpri_Rp_kWh + pvtarif_Rp_kWh +
#       west_max_flaeche + south_max_flaeche + east_max_flaeche | 
#       BFS_NUMMER,
#     data = df_train
# )
# df_test$pred_fe25c <- predict(fe25c, newdata = df_test)

# fe25d<- feols(
#     log(TotalPower) ~
#       elecpri_Rp_kWh + pvtarif_Rp_kWh +
#       west_max_flaeche + south_max_flaeche + east_max_flaeche | 
#       BFS_NUMMER + year,
#     data = df_train
# )
# df_test$pred_fe25d <- predict(fe25d, newdata = df_test)

# fe26a <- feols( log(TotalPower) ~  elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) + pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + west_max_flaeche + I(west_max_flaeche^2) + south_max_flaeche + I(south_max_flaeche^2) + east_max_flaeche + I(east_max_flaeche^2), data = df_train)
# fe26b <- feols( log(TotalPower) ~  elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) + pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) + west_max_flaeche + I(west_max_flaeche^2) + south_max_flaeche + I(south_max_flaeche^2) + east_max_flaeche + I(east_max_flaeche^2) | year, data = df_train )
# fe26c <- feols(
#     log(TotalPower) ~ 
#     elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
#     pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) +
#     west_max_flaeche + I(west_max_flaeche^2) +
#     south_max_flaeche + I(south_max_flaeche^2) +
#     east_max_flaeche + I(east_max_flaeche^2) |
#     BFS_NUMMER,
#     data = df_train
# )
# df_test$pred_fe26c <- predict(fe26c, newdata = df_test)

# fe26d <- feols(
#     log(TotalPower) ~ 
#     elecpri_Rp_kWh + I(elecpri_Rp_kWh^2) +
#     pvtarif_Rp_kWh + I(pvtarif_Rp_kWh^2) +
#     west_max_flaeche + I(west_max_flaeche^2) +
#     south_max_flaeche + I(south_max_flaeche^2) +
#     east_max_flaeche + I(east_max_flaeche^2) |
#     BFS_NUMMER + year,
#     data = df_train
# )
# df_test$pred_fe26d <- predict(fe26d, newdata = df_test)





In [ ]:
# %%R
# fe_model_list <- list(
#   fe11 = fe11, fe12 = fe12, fe13 = fe13, fe14 = fe14, 
#   fe21 = fe21, 
#   # fe22 = fe22, fe23 = fe23,
#   fe24a = fe24a, fe24b = fe24b, fe24c = fe24c, fe24d = fe24d, 
#   fe25a = fe25a, fe25b = fe25b, fe25c = fe25c, fe25d = fe25d,
#   fe26a = fe26a, fe26b = fe26b, fe26c = fe26c, fe26d = fe26d,
#   fe30 = fe30, fe31 = fe31,
#   fe40 = fe40, fe41 = fe41
# )

# # Export model info as a named list
# model_export <- lapply(fe_model_list, function(model) {
#   list(
#     comment = paste(deparse(formula(model)), collapse = " "),
#     coefficients = coef(model)
#   )
# })

# # Write JSON with model names as top-level keys
# write_json(
#   model_export,
#   path = file.path(getwd(), "data/calibration/calib_all_CH_bfs/reg2_fe_model_coef.json"),
#   pretty = TRUE,
#   auto_unbox = TRUE
# )

## ML random forest regression - full sample

In [ ]:
def run_random_forest_regression(rfr_mod_name, df_suffix, rfr_settings):

    # import
    df_train_rfr = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_train{df_suffix}.csv')
    df_test_rfr  = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_test{df_suffix}.csv')

    old_pred_cols = [col for col in df_test_rfr.columns if 'pred_' in col]
    df_test_rfr_old = df_test_rfr[old_pred_cols + ['EGID', ]].copy()


    # transformations
    # df_train_rfr = df_train_rfr.drop(columns=['EGID', 'xtf_id', 'n_DF_UID']).copy()
    cols_dtypes_tupls = {
        # 'year': 'int64',
        'BFS_NUMMER': 'category',
        'GAREA': 'float64',
        # 'GBAUJ': 'int64',   
        'GKLAS': 'category',
        # 'GSTAT': 'category',
        'GWAERZH1': 'category',
        'GENH1': 'category',
        'GWAERZH1_str': 'category',
        # 'InitialPower': 'float64',
        'TotalPower': 'float64',
        'elecpri_Rp_kWh': 'float64',
        'pvtarif_Rp_kWh': 'float64',
        'FLAECHE_total': 'float64',
        'east_max_flaeche': 'float64',
        'west_max_flaeche': 'float64',
        'north_max_flaeche': 'float64',
        'south_max_flaeche': 'float64',
    }
    df_train_rfr = df_train_rfr[[col for col in cols_dtypes_tupls.keys() if col in df_train_rfr.columns]].copy()
    df_test_rfr = df_test_rfr[[col for col in cols_dtypes_tupls.keys() if col in df_test_rfr.columns]].copy()

    df_train_rfr = df_train_rfr.dropna().copy()
    df_test_rfr = df_test_rfr.dropna().copy()


    for col, dtype in cols_dtypes_tupls.items():
        df_train_rfr[col] = df_train_rfr[col].astype(dtype)
        df_test_rfr[col]  = df_test_rfr[col].astype(dtype)

    X = df_train_rfr.drop(columns=['TotalPower', ])
    y = df_train_rfr['TotalPower']

    # encode categorical variables
    cat_cols = X.select_dtypes(include=["object", "category"]).columns
    cat_cols = X.select_dtypes(include=["object", "category"]).columns
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_Arry = encoder.fit_transform(X[cat_cols].astype(str))
    encoded_df = pd.DataFrame(encoded_Arry, columns=encoder.get_feature_names_out(cat_cols))
    X = pd.concat([X.drop(columns=cat_cols).reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)


    # rf model
    if rfr_settings['run_ML_rfr_TF']:
        # rfr_model = RandomForestRegressor(
        #     n_estimators   = rfr_settings['n_estimators'],
        #     max_depth      = rfr_settings['max_depth'],
        #     random_state   = rfr_settings['random_state'],
        #     n_jobs         = rfr_settings['n_jobs'],
        # )
        # # cross validation
        # kf = KFold(n_splits=10, shuffle=True, random_state=42)
        # cv_scores = cross_val_score(rfr_model, X, y, cv=kf, scoring='neg_mean_absolute_error')
        # rfr_model.fit(X, y)

        rfr_model = RandomForestRegressor(random_state = rfr_settings['random_state'])
        param_grid = {
            'n_estimators':      rfr_settings['n_estimators'],
            'min_samples_split': rfr_settings['min_samples_split'],
            'max_depth':         rfr_settings['max_depth'],
        }
            
        grid_search = GridSearchCV(
            rfr_model,
            param_grid,
            cv=rfr_settings['cross_validation'],
            scoring='neg_mean_absolute_error',
            n_jobs=rfr_settings['n_jobs'],
            return_train_score=True,
        )
        grid_search.fit(X, y)
        rfr_model = grid_search.best_estimator_

        # save model
        joblib.dump(rfr_model, f'{name_dir_export_path_PY}/reg2_{rfr_mod_name}_model.pkl')
        joblib.dump(encoder, f'{name_dir_export_path_PY}/reg2_{rfr_mod_name}_encoder.pkl')


    # prediction
    if rfr_settings['run_ML_rfr_TF']:
        X_test = df_test_rfr.drop(columns=['TotalPower', ])
        encoded_test_array = encoder.transform(X_test[cat_cols].astype(str))
        encoded_test_df = pd.DataFrame(encoded_test_array, columns=encoder.get_feature_names_out(cat_cols))

        X_test_final = pd.concat([X_test.drop(columns=cat_cols).reset_index(drop=True), encoded_test_df.reset_index(drop=True)], axis=1)
        X_test_final = X_test_final[X.columns]

        test_preds = rfr_model.predict(X_test_final)

        df_test_rfr[f'pred_{rfr_mod_name}'] = test_preds

    else: 
        df_test_rfr[f'pred_{rfr_mod_name}'] = np.zeros(df_test_rfr.shape[0])

    df_test_rfr.to_csv(f'{name_dir_export_path_PY}/reg2_df_test{df_suffix}_RFR.csv', index=False)

    del df_train_rfr, df_test_rfr


In [ ]:
run_ML_rfr_TF = True
rfr_settings = {
    'comment_settings_RandomForestRegressor()': 
        """
        n_estimators:       Defines the number of decision trees in the Random Forest.
        random_state=0:     Ensures the randomness in model training is controlled for reproducibility.
        oob_score=True:     Enables out-of-bag scoring which evaluates the model's performance using data 
                            not seen by individual trees during training
        max_depth:          The maximum depth of the tree. If None, then nodes are expanded until all 
                            leaves are pure or until all leaves contain less than min_samples_split 
                            samples

    """,
    'run_ML_rfr_TF':        True,
    'random_state':         None,    # default: None  # | None,    
    'n_jobs':               -1,      # default: None  # | -1,      
    'cross_validation':     2, 
    'n_estimators':         [100, ]  ,    # default: 100   # | 1,       
    'min_samples_split':    [5, ]    ,    # default: 2     # | 1000,    
    'max_depth':            [20, ]   ,    # default: None  # | 3,       
}

In [ ]:
# run_random_forest_regression(rfr_mod_name='local_nb_rfr1', df_suffix='',             rfr_settings=rfr_settings)
# run_random_forest_regression(rfr_mod_name='local_nb_rfr2', df_suffix='_kwpmax20',    rfr_settings=rfr_settings)

## GAM - Generalized Additive Models

In [ ]:
# df_train = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_train.csv')
# df_test = pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_test.csv')

# features = ['elecpri_Rp_kWh', 'pvtarif_Rp_kWh',
#             'north_max_flaeche', 'south_max_flaeche',
#             'east_max_flaeche', 'west_max_flaeche']

# x_train = df_train[features].values
# y_train = df_train['TotalPower'].values

# x_test = df_test[features].values
# y_test = df_test['TotalPower'].values

# gam1 = LinearGAM(
#     s(0) + s(1) + s(2) + s(3) + s(4) + s(5)
# ).fit(x_train, y_train)
# y_pred_gam1 = gam1.predict(x_train)
# df_test['pred_gam1'] = gam1.predict(x_test)

# # export coefs and df_test
# model_formula = "TotalPower ~ s(elecpri_Rp_kWh) + s(pvtarif_Rp_kWh) + s(north_max_flaeche) + s(south_max_flaeche) + s(east_max_flaeche) + s(west_max_flaeche)"
# coefficients = gam1.coef_.tolist()
# lambdas = list(gam1.lam)
# terms = [str(gam1.terms[i]) for i in range(len(gam1.terms))]
# reg2_gam_model_coef = {
#     "comment": model_formula,
#     "coefficients": coefficients,
#     "lambdas": lambdas,
#     "terms": terms
# }
# with open(f'{name_dir_export_path_PY}/reg2_gam_model_coef.json', 'w') as f:
#     json.dump(reg2_gam_model_coef, f, indent=4)

# df_test.to_csv(f'{name_dir_export_path_PY}/reg2_df_test.csv', index=False)



In [ ]:
# df_train_res1to2 =  pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_train_res1to2.csv')
# df_test_res1to2 =   pd.read_csv(f'{name_dir_export_path_PY}/reg2_df_test_res1to2.csv' )

# features = ['elecpri_Rp_kWh', 'pvtarif_Rp_kWh',
#             'north_max_flaeche', 'south_max_flaeche',
#             'east_max_flaeche', 'west_max_flaeche']

# x_train = df_train_res1to2[features].values
# y_train = df_train_res1to2['TotalPower'].values

# x_test = df_test_res1to2[features].values
# y_test = df_test_res1to2['TotalPower'].values

# gam2 = LinearGAM(
#     s(0) + s(1) + s(2) + s(3) + s(4) + s(5)
# ).fit(x_train, y_train)
# y_pred_gam2 = gam2.predict(x_train)
# df_test_res1to2['pred_gam2'] = gam2.predict(x_test)

# # export coefs and df_test
# model_formula = "TotalPower ~ s(elecpri_Rp_kWh) + s(pvtarif_Rp_kWh) + s(north_max_flaeche) + s(south_max_flaeche) + s(east_max_flaeche) + s(west_max_flaeche)"
# coefficients = gam2.coef_.tolist()
# lambdas = list(gam2.lam)
# terms = [str(gam2.terms[i]) for i in range(len(gam2.terms))]
# reg2_gam_model_coef = {
#     "comment": model_formula,
#     "coefficients": coefficients,
#     "lambdas": lambdas,
#     "terms": terms
# }
# with open(f'{name_dir_export_path_PY}/reg2_gam_model_coef.json', 'w') as f:
#     json.dump(reg2_gam_model_coef, f, indent=4)

# df_test_res1to2.to_csv(f'{name_dir_export_path_PY}/reg2_df_test_res1to2.csv', index=False)


## visualization

In [16]:
# PREDICTION ACCURACY PLOTS
if True: 
    pred_colname_dfname_tuples = [
        ('pred_fe21',              'reg2_df_test.csv'),
        ('pred_fe31',              'reg2_df_test_res3plus.csv'),
        ('pred_fe32',              'reg2_df_test_res1to2.csv'),
        # ('pred_fe40',              'reg2_df_test_res3plus.csv'),
        # ('pred_fe41',              'reg2_df_test_res1to2.csv'),
        # ('pred_local_nb_rfr1',     'reg2_df_test_RFR.csv'),
        # ('pred_local_nb_rfr2',     'reg2_df_test_kwpmax20_RFR.csv'),
        # ('opt_kWp_s0-00',       'df_test_optim_s0-00.csv' ),
        # ('opt_kWp_s0-20',       'df_test_optim_s0-20.csv' ),
        # ('opt_kWp_s0-30',       'df_test_optim_s0-30.csv' ),
        ('pred_rfr2b',          'df_test_pvroof20to70.csv'), 


                                
        
    ]
    # accuracy plot ------------
    pred_cols = [tup[0] for tup in pred_colname_dfname_tuples]
    ncols = 3
    nplots = len (pred_cols)
    nrows = math.ceil(nplots / ncols)
    fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=pred_cols, horizontal_spacing=0.1, vertical_spacing=0.15)

    for i, (pred_col, df_name) in enumerate(pred_colname_dfname_tuples):
        row = (i // ncols) + 1
        col = (i % ncols) + 1
        print(f'path:{name_dir_export_path_PY}/{df_name}')
        print(f'os.exists: {os.path.exists(f"{name_dir_export_path_PY}/{df_name}")}')

        df_import = pd.read_csv(f'{name_dir_export_path_PY}/{df_name}')
        totalpower_col = 'TotalPower_W' if 'TotalPower_W' in df_import.columns else 'TotalPower'

        df_import = df_import.dropna(subset=[pred_col, totalpower_col])    
        df_accu = df_import.replace([np.inf, -np.inf], np.nan).dropna(subset=[ totalpower_col, pred_col]).copy()
        
        fig.add_trace(
            go.Scatter(
                x=df_accu[pred_col],
                y=df_accu[totalpower_col],
                mode='markers',
                marker=dict(size=3, opacity=0.5),
                name=pred_col,
                showlegend=False
            ),
            row=row,
            col=col
        )
        # add diagonal line y=x
        min_val = min(df_accu[totalpower_col].min(), df_accu[pred_col].min()) * 0.95
        max_val = max(df_accu[totalpower_col].max(), df_accu[pred_col].max()) * 1.05
        fig.add_trace(go.Scatter(x=[min_val, max_val],
                                    y=[min_val, max_val],
                                    mode='lines',
                                    line=dict(color='red', dash='dash'),
                                    name='Perfect Prediction',
                                    showlegend=False),
                        row=row, col=col)

        fig.update_xaxes(title_text=f'PredPwr {df_accu.shape[0]}n', row=row, col=col)
        fig.update_yaxes(title_text='Total Power', row=row, col=col)

    # layout 
    fig.update_layout(
        # height=300 * nrows, width=1000,
                    title_text=f'Predicted Power vs Total Power ({df_accu.shape[0]} n_df_train, {df_accu.shape[0]} n_df_test)',
                    plot_bgcolor='white')
    for i in range(1, nplots + 1):
        row = (i - 1) // ncols + 1
        col = (i - 1) % ncols + 1
        # fig.update_xaxes(title_text='Predicted Power', row=row, col=col)
        # fig.update_yaxes(title_text='Total Power', row=row, col=col)

    fig.write_html(f'{name_dir_export_path_PY}/reg2_pred_vs_total_power.html')


    # RMSE plot ------------
    rmse_dict = {}

    for i, (pred_col, df_name) in enumerate(pred_colname_dfname_tuples):

        df_import = pd.read_csv(f'{name_dir_export_path_PY}/{df_name}') 
        totalpower_col = 'TotalPower_W' if 'TotalPower_W' in df_import.columns else 'TotalPower'

        df_import = df_import.dropna(subset=[pred_col, totalpower_col])    
        df_accu = df_import.replace([np.inf, -np.inf], np.nan).dropna(subset=[totalpower_col, pred_col]).copy()
        
        rmse = mean_squared_error(df_accu[totalpower_col], df_accu[pred_col])
        rmse_dict[pred_col] = rmse

    bar_fig = go.Figure()

    bar_fig.add_trace(
        go.Bar(
            x=list(rmse_dict.keys()),
            y=list(rmse_dict.values()),
            marker=dict(color='cornflowerblue'),
            text=[f"{v:.2f}" for v in rmse_dict.values()],
            textposition='auto'
        )
    )

    bar_fig.update_layout(
        title='RMSE of Model Predictions on Test Set',
        xaxis_title='Model',
        yaxis_title='RMSE',
        plot_bgcolor='white'
    )

    # Save bar plot
    bar_fig.write_html(f'{name_dir_export_path_PY}/reg2_rmse_barplot.html')




path:c:/Models/OptimalPV_RH/data/calibration/calib_all_CH_bfs3/reg2_df_test.csv
os.exists: True
path:c:/Models/OptimalPV_RH/data/calibration/calib_all_CH_bfs3/reg2_df_test_res3plus.csv
os.exists: True
path:c:/Models/OptimalPV_RH/data/calibration/calib_all_CH_bfs3/reg2_df_test_res1to2.csv
os.exists: True
path:c:/Models/OptimalPV_RH/data/calibration/calib_all_CH_bfs3/df_test_pvroof20to70.csv
os.exists: True


KeyError: ['pred_rfr2b']